In [1]:
import calendar
import os
import pandas as pd
import sys
from datetime import datetime
from sqlalchemy.sql import extract
from collections import defaultdict, OrderedDict
from sqlalchemy import create_engine
caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_virasana)
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from bhadrasana.models import engine, db_session
from bhadrasana.models import Usuario
from bhadrasana.models.ovr import OVR, EventoOVR
from bhadrasana.models.rvfmanager import lista_rvfovr

2023-05-17 11:42:36,912 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\access.log


C:\Users\25052288840\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def get_fichas_portipo(ano, mes):
    lista_ovrs = db_session.query(OVR).join(EventoOVR)\
                                      .filter(OVR.setor_id.in_((1, 2, 3)))\
                                      .filter(EventoOVR.fase >= 3)\
                                      .filter(extract('year', EventoOVR.create_date) ==  ano)\
                                      .filter(extract('month', EventoOVR.create_date) ==  mes)\
                                      .order_by(OVR.tipooperacao, OVR.id).all()
    lista_fichas = set()
    for ovr in lista_ovrs:
        lista_fichas.add(ovr)
    return lista_fichas

In [3]:
def preenche_linha_ovr(operacao, year, month):
    lastday = calendar.monthrange(year, month)[1]
    ano = str(year)
    mes = str(month).zfill(2)
    inicio = f'01/{mes}/{ano}'
    termino = f'{lastday}/{mes}/{ano}'
    campos_ovr = OrderedDict()
    NOMES_TIPOOPERACAO = {
        1: f'IMPORTAÇÃO - OPERAÇÃO LUNETA {ano}{mes}',
        # Observação: ideal seria dividir mais as OVRs, pois além de Operação Luneta, tivemos Outlet, Ramenta, temos a seleção de 
        # bagagem, etc. Mas aí precisaria ser "Hard-coded" o agrupamento por OVRs
        2: f'EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO {ano}{mes}',
        # Observação: hoje não temos, mas se fizermos OUTROS TIPOS DE OPERAÇÕES em exportação que não a habitual busca de cocaína,
        # o ideal seria separar em outras OVRs
        3: f'IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO LUNETA {ano}{mes}',
        # Observação: ideal seria dividir mais as OVRs, pois além de Operação Luneta, tivemos Outlet, Ramenta, temos a seleção de 
        # bagagem, etc. Mas aí precisaria ser "Hard-coded" o agrupamento por OVRs
        4: f'EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO PORTO BLINDADO {ano}{mes}',
        # Observação: existe também o código "Pós-incidente" e não temos distinção no Fichas, vamos criar um Flag.
        # no caso deste flag, substituir estes campos por 'Tipo': 'Pós-Incidente - Portos', 'Código': '30.02.10'
        7: f'VIGILÂNCIA - OPERAÇÃO REDOMA {ano}{mes}',
    }
    campos_ovr['Nome'] = NOMES_TIPOOPERACAO[operacao.tipooperacao]
    if operacao.tipooperacao in (1, 2):
        campos_ovr['Modalidade'] = 'Vigilância e Repressão'
        campos_ovr['Tipo'] = 'Portos'
        campos_ovr['Código'] = '20.00.10'
    elif operacao.tipooperacao in (3, 4):
        campos_ovr['Modalidade'] = 'Pesquisa e Seleção'
        campos_ovr['Tipo'] = 'Pré-Incidente - Portos'
        campos_ovr['Código'] = '30.01.10'
    elif operacao.tipooperacao == 7:
        campos_ovr['Modalidade'] = 'Vigilância e Repressão'
        # Observação: Secta e-OVR não possui a tipificação de vigilância/rondas, auditorias e conformidade
        # Utilizando este tipo e código para distinção
        campos_ovr['Tipo'] = 'Ponto de fronteira'
        campos_ovr['Código'] = '20.00.09'
    else:
        raise Exception(f'tipooperacao {operacao.tipooperacao} não previsto!!')
    campos_ovr['Abrangência'] = 'Local'
    campos_ovr['Início'] = inicio
    campos_ovr['Término'] = termino
    campos_ovr['Local da Execução'] = 'Porto de Santos'
    campos_ovr['Coordenador'] = 'Ivan da Silva Brasílico'
    campos_ovr['Coordenador_CPF'] = '25052288840'
    campos_ovr['Motivação'] = 'Demanda Interna'
    # Por último, analisa Flags para mudar Descrição da Operação
    if len(operacao.flags) > 0:
        flags = [flag for flag in operacao.flags]
        for flag in flags:
            if 'Operação' in flag.nome:
                campos_ovr['Nome'] = flag.nome + f' {ano}{mes}'
                break
            if flag.nome == 'Pós incidente':
                campos_ovr['Nome'] = 'Pós-Incidente - ' + campos_ovr['Nome']
                campos_ovr['Tipo'] = 'Pós-Incidente - Portos'
                campos_ovr['Código'] = '30.02.10'
    return campos_ovr

In [4]:
def get_peso_apreensoes(session, rvfs):
    peso = 0.
    for rvf in rvfs:
        for apreensao in rvf.apreensoes:
            try:
                peso += float(apreensao.peso)
            except TypeError:
                pass
    return peso

def get_valor_tgs(ovr):
    valor = 0.
    for tg in ovr.tgs:
        try:
            valor += float(tg.valor)
        except TypeError:
            pass
    return valor


def preenche_linha_operacao(operacao, modalidade):
    linha = OrderedDict()
    linha['Órgão'] = 'RFB'
    linha['Equipe'] = operacao.setor.nome
    linha['Modalidade Operação'] = modalidade
    linha['Tipo'] = 'Portos'
    linha['Local da Operação'] = operacao.recinto.nome
    linha['Documentos de origem'] = f'Ficha {operacao.id}'
    linha['Ocorrências durante a operação'] = operacao.observacoes
    linha['Supervisor da RFB'] = str(operacao.cpfauditorresponsavel)
    linha['Responsável'] = str(operacao.responsavel_cpf)
    rvfs = lista_rvfovr(db_session, operacao.id)
    valortgs = get_valor_tgs(operacao)
    pesoapreensoes = get_peso_apreensoes(db_session, rvfs)
    nhoras = len(rvfs) * 8
    if valortgs > 0:
        nhoras = nhoras * 5
    if pesoapreensoes > 0:
        nhoras = nhoras * 2
    servidores_set = set()
    for evento in operacao.historico:
        servidores_set.add(evento.user_name)
    servidores = []
    for cpf in servidores_set:
        servidor = db_session.query(Usuario).filter(Usuario.cpf == cpf).one()
        servidores.append({'cpf': servidor.cpf, 'nome': servidor.nome, 'horas': nhoras})
    linha['Servidores'] = servidores
    linha['Perdimento'] = valortgs
    linha['Apreensao'] = pesoapreensoes
    return linha


In [5]:
mes = 5
ano = 2023

lista_fichas = get_fichas_portipo(ano, mes)

In [6]:
lista_final = []
for ficha in lista_fichas:
    try:
        linha_ovr = preenche_linha_ovr(ficha, ano, mes)
        linha_operacao = preenche_linha_operacao(ficha, linha_ovr['Modalidade'])
        linha_final = linha_ovr.copy()
        linha_final.update(linha_operacao)
        lista_final.append(linha_final)
    except Exception as err:
        print(err)
        continue


0
5
6
6


In [7]:
df = pd.DataFrame(lista_final)
df = df.sort_values('Nome')

In [8]:
df

,Nome,Modalidade,Tipo,Código,Abrangência,Início,Término,Local da Execução,Coordenador,Coordenador_CPF,...,Equipe,Modalidade Operação,Local da Operação,Documentos de origem,Ocorrências durante a operação,Supervisor da RFB,Responsável,Servidores,Perdimento,Apreensao
94,EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,Pesquisa e Seleção,Portos,30.01.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Pesquisa e Seleção,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 5910,Seleção feita com base na imagem de escâner\r\...,01126891754,01126891754,"[{'cpf': '01126891754', 'nome': 'Fernando Nogu...",0.0,0.0
54,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 6038,Seleção feita pela equipe COV/DIREP/ALF - SANT...,01126891754,01126891754,"[{'cpf': '01126891754', 'nome': 'Fernando Nogu...",0.0,0.0
35,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,Brasil Terminal Portuario,Ficha 6028,04.793.761/0001-07 - RSX COMERCIAL TRANSPORTES...,01126891754,01126891754,"[{'cpf': '01126891754', 'nome': 'Fernando Nogu...",0.0,0.0
36,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,EMBRAPORT - EMPRESA BRASILEIRA DE TERMINAIS PO...,Ficha 6129,Seleção feita pela equipe COV/DIREP/ALF - SANT...,01126891754,01126891754,"[{'cpf': '01126891754', 'nome': 'Fernando Nogu...",0.0,0.0
93,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 6051,07.092.088/0001-68 - FRANCFORT COMERCIO IMPORT...,01126891754,01126891754,"[{'cpf': '01126891754', 'nome': 'Fernando Nogu...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,VIGILÂNCIA - OPERAÇÃO REDOMA 202305,Vigilância e Repressão,Portos,20.00.09,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ALFANDEGA,Ficha 6066,Dia 17/04\r\nRonda no canal do porto e operaçã...,None,None,"[{'cpf': '00579600629', 'nome': 'Alysson Ribei...",0.0,0.0
64,VIGILÂNCIA - OPERAÇÃO REDOMA 202305,Vigilância e Repressão,Portos,20.00.09,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ALFANDEGA,Ficha 6067,Dia 18/04\r\nRonda no canal do porto\r\nTripul...,None,None,"[{'cpf': '00579600629', 'nome': 'Alysson Ribei...",0.0,0.0
69,VIGILÂNCIA - OPERAÇÃO REDOMA 202305,Vigilância e Repressão,Portos,20.00.09,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ALFANDEGA,Ficha 6068,Dia 19/04\r\nRonda no canal do porto.\r\nTripu...,None,None,"[{'cpf': '00579600629', 'nome': 'Alysson Ribei...",0.0,0.0
31,VIGILÂNCIA - OPERAÇÃO REDOMA 202305,Vigilância e Repressão,Portos,20.00.09,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP,Vigilância e Repressão,EUDMARCO - PORTO SECO,Ficha 3585,None,25052288840,25052288840,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva...",0.0,0.0


In [9]:
df_ovrs = df.groupby(df.Nome).agg(
    quantidade=pd.NamedAgg(column='Documentos de origem', aggfunc='count'),
    perdimentos=pd.NamedAgg(column='Perdimento', aggfunc='sum'),
    apreensoes=pd.NamedAgg(column='Apreensao', aggfunc='sum'),
).reset_index()
df_ovrs

,Nome,quantidade,perdimentos,apreensoes
0,EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,1,0.0,0.00
1,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202305,60,0.0,1892.79
2,IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,7,0.0,0.00
3,IMPORTAÇÃO - OPERAÇÃO LUNETA 202305,4,717875.0,0.00
4,Operação Outlet 202305,1,641562.5,0.00
5,Operação Ramenta 202305,3,0.0,0.00
6,VIGILÂNCIA - OPERAÇÃO REDOMA 202305,33,0.0,0.00


In [10]:
df.to_excel('teste.xlsx')

In [11]:
df.Nome.value_counts()

EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202305                                    60
VIGILÂNCIA - OPERAÇÃO REDOMA 202305                                            33
IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO LUNETA 202305             7
IMPORTAÇÃO - OPERAÇÃO LUNETA 202305                                             4
Operação Ramenta 202305                                                         3
EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO PORTO BLINDADO 202305     1
Operação Outlet 202305                                                          1
Name: Nome, dtype: int64

In [29]:
df.iloc[0][1:12]

Nome                 EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...
Modalidade                                          Pesquisa e Seleção
Tipo                                                            Portos
Código                                                        30.01.10
Abrangência                                                      Local
Início                                                      01/05/2023
Término                                                     31/05/2023
Local da Execução                                      Porto de Santos
Coordenador                                    Ivan da Silva Brasílico
Coordenador_CPF                                            25052288840
Motivação                                              Demanda Interna
Name: 0, dtype: object

In [18]:
pd.options.display.float_format = '{:,.2f}'.format
df[df.Nome=="Operação Outlet 202305"]

,Unnamed: 0,Nome,Modalidade,Tipo,Código,Abrangência,Início,Término,Local da Execução,Coordenador,...,Equipe,Modalidade Operação,Local da Operação,Documentos de origem,Ocorrências durante a operação,Supervisor da RFB,Responsável,Servidores,Perdimento,Apreensao
72,4,Operação Outlet 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,...,ALFSTS/DIREP/EQRIMP,Vigilância e Repressão,BANDEIRANTES DEICMAR - CLIA ALEMOA,Ficha 2268,Operação Outlet97._x000D_\nCE-Mercante 1521051...,06806235824,06806235824,"[{'cpf': '06806235824', 'nome': 'Maristela Cor...","641,562.50",0.00


In [13]:
df = pd.read_excel('teste.xlsx')

In [14]:
df.Nome.value_counts()

EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202305                                    60
VIGILÂNCIA - OPERAÇÃO REDOMA 202305                                            33
IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO LUNETA 202305             7
IMPORTAÇÃO - OPERAÇÃO LUNETA 202305                                             4
Operação Ramenta 202305                                                         3
EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO PORTO BLINDADO 202305     1
Operação Outlet 202305                                                          1
Name: Nome, dtype: int64

In [15]:
df[df.Nome=="Operação Ramenta 202305"]

,Unnamed: 0,Nome,Modalidade,Tipo,Código,Abrangência,Início,Término,Local da Execução,Coordenador,...,Equipe,Modalidade Operação,Local da Operação,Documentos de origem,Ocorrências durante a operação,Supervisor da RFB,Responsável,Servidores,Perdimento,Apreensao
73,43,Operação Ramenta 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,...,ALFSTS/DIREP,Vigilância e Repressão,BANDEIRANTES DEICMAR - CLIA ALEMOA,Ficha 2977,NaN,25052288840,25052288840,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva...",0.0,0.0
74,39,Operação Ramenta 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,SANTOS BRASIL LOGISTICA S/A - CLIA GUARUJA,Ficha 2919,NaN,25052288840,25052288840,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva...",0.0,0.0
75,38,Operação Ramenta 202305,Vigilância e Repressão,Portos,20.00.10,Local,01/05/2023,31/05/2023,Porto de Santos,Ivan da Silva Brasílico,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,SANTOS BRASIL LOGISTICA S/A - CLIA SANTOS,Ficha 2801,Operação Ramenta_x000D_\n_x000D_\nContainers c...,25052288840,25052288840,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva...",0.0,0.0
